In [1]:
import pandas as pd
def get_hdf_keys(dir):
    with pd.HDFStore(dir) as store:
        return [e[1:] for e in store.keys()]

dir = 'danawa_crawling_data.h5'
print(get_hdf_keys(dir))

['CPU', 'CPU_detail', 'Case', 'Cooler', 'Earphone', 'HDD', 'Headphone', 'Headset', 'Keyboard', 'Laptop', 'MBoard', 'Monitor', 'Mouse', 'Power', 'RAM', 'RAM_detail', 'SSD', 'Speaker', 'VGA', 'VGA_detail']


In [3]:
df = pd.read_hdf(dir, 'CPU')
df

,id_validator
id,
12621851,True
12625265,True
15137684,True
13763867,True
12872324,True
...,...
2322793,True
1220718,True
686573,True


In [33]:
df1 = pd.read_hdf('danawa_crawling_data.h5', 'MBoard')
df2 = pd.read_hdf('danawa_crawling_data_old.h5', 'MBoard')
df3 = pd.read_hdf('danawa_crawling_data_old2.h5', 'MBoard')
s1 = set(df1.index)
s2 = set(df2.index)
s3 = set(df3.index)
print(len(s1))
print(len(s2))
print(len(s3))
print(s3 - s2)
print(s2 - s3)
print(s1 - s2)
print(s2 - s1)
print(s1 - s3)
print(s3 - s1)
print(list(df3.index).index('11231361') // 90, list(df3.index).index('11231361') % 90)

1723
1726
1727
{'11231361', '9955284'}
{'6053634'}
{'11231361', '9955284'}
{'15477053', '6053634', '10006614', '13434644', '16526279'}
set()
{'10006614', '16526279', '13434644', '15477053'}
14 79


In [45]:
import pandas as pd
def get_hdf_keys(dir):
    with pd.HDFStore(dir) as store:
        return [e[1:] for e in store.keys()]

dir = 'danawa_crawling_data_old.h5'
print(get_hdf_keys(dir))
df = pd.read_hdf(dir, 'SSD_review')
df

['CPU', 'Case', 'Cooler', 'Earphone', 'HDD', 'Headphone', 'Headset', 'Keyboard', 'MBoard', 'Monitor', 'Mouse', 'Power', 'RAM', 'SSD', 'SSD_detail', 'SSD_review', 'Speaker', 'VGA']


,id,sentence,time,good,bad
0,13190711,QLC로 토렌트 셰어용으로 몇달 쓰니까 사요나라임.. 최소 TLC..,2022.07.07. 16:48:14,,
1,13190711,다행히 아직 TLC군요,2021.06.28. 14:22:56,1,
2,13190711,870 QVO 8TB 와 고민하다가... 870 EVO 4TB 로 결정했어요. 사실...,2021.03.23. 13:28:33,2,
3,13190711,S-ATA 방식의 마지막 작품이려나,2021.02.02. 04:55:17,,
4,13190711,2년뒤에 블프에 사야지...,2021.01.26. 13:44:19,,
...,...,...,...,...,...
3527,11127390,예전에 ADATA 다른 M2 제품은 반대쪽이 평판이라\n이것도 그런줄 알고 샀더니 ...,2020.08.20. 15:15:46,,
3528,10927932,노트북 구입하면서 이 SSD 두 개를 사은품으로 받았는데 절대로 쓰면 안되는 SSD...,2022.03.26. 19:44:26,,
3529,10927932,기가바이트 B250M-DS3H 메인보드에서 사용 가능한가요?,2020.09.10. 09:05:47,1,
3530,10927923,디램없는데 구지 이걸 사서 고생할 필요는 없을듯 ㅎ,2021.12.11. 08:33:33,,
